In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2    
from sklearn.metrics import f1_score
import glob
from keras.preprocessing import image
from keras.models import Sequential, Model
from keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D,AveragePooling2D, Flatten, Dense, Dropout, Activation , Concatenate, Input , BatchNormalization, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator ,img_to_array, load_img
from keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping
from keras.optimizers import SGD
from keras.utils import plot_model
from keras import Model
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
df = pd.read_csv('../input/nikitapupu/data (1).csv')

In [3]:
nd = [0]*50000
nr = [1]*25001
ny = [2]*25001
nt = nd+nr+ny

In [4]:
datar = pd.DataFrame(nt, columns=['partition'])

In [5]:
df_par_attr = df.join(datar)

In [6]:
df_par_attr.set_index('path', inplace=True)

In [7]:
df_par_attr

,label,partition
path,,
male/085992.jpg,0,0
male/046332.jpg,0,0
female/097359.jpg,1,0
female/170442.jpg,1,0
male/008056.jpg,0,0
...,...,...
female/174616.jpg,1,2
female/177533.jpg,1,2
male/095538.jpg,0,2


In [8]:
images_folder = "../input/ntechdata/internship_data/"
def load_reshape_img(fname):
    img = load_img(fname, target_size = (224, 224), color_mode='rgb')
    x = img_to_array(img)/255.
    x = x.reshape((1,) + x.shape)

    return x

from keras.utils import np_utils

def generate_df(partition, attr, num_samples):
    '''
    partition
        0 -> train
        1 -> validation
        2 -> test
    
    '''
    
    df_ = df_par_attr[(df_par_attr['partition'] == partition) 
                           & (df_par_attr[attr] == 0)].sample(int(num_samples/2))
    df_ = pd.concat([df_,
                      df_par_attr[(df_par_attr['partition'] == partition) 
                                  & (df_par_attr[attr] == 1)].sample(int(num_samples/2))])

    # for Train and Validation
    if partition != 2:
        x_ = np.array([load_reshape_img(images_folder + fname) for fname in df_.index])
        x_ = x_.reshape(x_.shape[0], 224, 224, 3)
        y_ = np_utils.to_categorical(df_[attr],2)
    # for Test
    else:
        x_ = []
        y_ = []

        for index, target in df_.iterrows():
            im = cv2.imread(images_folder + index)
            im = cv2.resize(cv2.cvtColor(im, cv2.COLOR_BGR2RGB), (IMG_WIDTH, IMG_HEIGHT)).astype(np.float32) / 255.0
            im = np.expand_dims(im, axis =0)
            x_.append(im)
            y_.append(target[attr])

    return x_, y_

In [9]:
TRAINING_SAMPLES = 10000
VALIDATION_SAMPLES = 2500
TEST_SAMPLES = 2500
IMG_WIDTH = 224
IMG_HEIGHT = 224
BATCH_SIZE = 16
NUM_EPOCHS = 4

In [10]:
from keras.applications.resnet50 import ResNet50, preprocess_input
x_train, y_train = generate_df(0, 'label', TRAINING_SAMPLES)
images_folder = "../input/ntechdata/internship_data/"
# Train - Data Preparation - Data Augmentation with generators
train_datagen =  ImageDataGenerator(
  preprocessing_function=preprocess_input,
  rotation_range=30,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True,
)

train_datagen.fit(x_train)

train_generator = train_datagen.flow(
x_train, y_train,
batch_size=BATCH_SIZE,
)

In [11]:
x_valid, y_valid = generate_df(1, 'label', VALIDATION_SAMPLES)

In [12]:
inc_model = ResNet50(weights='imagenet',
                        include_top=False,
                        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

print("number of layers:", len(inc_model.layers))
#inc_model.summary()

94773248/94765736 [==============================] - 2s 0us/step
number of layers: 175


In [13]:
#Adding custom Layers
x = inc_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(512, activation="relu")(x)
predictions = Dense(2, activation="softmax")(x)

In [14]:
model_ = Model(inputs=inc_model.input, outputs=predictions)

In [17]:
model_.load_weights('../input/jhlsfg/Scurt_best_1.hdf5')

In [18]:
# creating the final model 




# Lock initial layers to do not be trained
for layer in model_.layers[:52]:
    layer.trainable = False

# compile the model

model_.compile(optimizer=SGD(lr=0.0001, momentum=0.9)
                    , loss='binary_crossentropy'
                    , metrics=['accuracy'])

In [ ]:
checkpoint_path = "../input/durtuso"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [19]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='orty.best.hdf5', 
                               verbose=1, save_weights_only=True, period = 1)

In [ ]:
hist = model_.fit_generator(train_generator
                     , validation_data = (x_valid, y_valid)
                      , steps_per_epoch= TRAINING_SAMPLES/BATCH_SIZE
                      , epochs= NUM_EPOCHS
                      , callbacks=[checkpointer]
                      , verbose=1
                            
                    )

Epoch 1/4
